<a href="https://colab.research.google.com/github/urlapovia03/university/blob/main/%D0%94%D0%9F%D0%9E_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_5_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №4. Введение в Python. ООП рефакторинг**

---




**Обучающийся:** *[Урлапов Иван Алексеевич]*  



---

## **Цель работы:**



Провести рефакторинг ранее созданного Python-пакета в предыдущей практической работе, преобразовав его функциональность в соответствии с принципами объектно-ориентированного программирования. Обновленный пакет должен включать классы и методы для преобразования координат между декартовой и сферической системами координат, а также для работы с файлами.



## **Задачи:**



1. Анализ существующего пакета и определение необходимых изменений для перехода на ООП-парадигму.

2. Создание новой структуры пакета с использованием классов и модулей, соответствующих принципам ООП.

3. Реализация классов и методов для преобразования координат между декартовой и сферической системами координат.

4. Реализация классов и методов для работы с файлами, обеспечивая удобный интерфейс для чтения и записи данных.

5. Создание файла `__main__.py` с консольным интерфейсом для взаимодействия с функциональностью пакета.

6. Тестирование и проверка работоспособности обновленного пакета на примерах, подтверждение корректности реализации.



## **Демонстрация результата:**



Вставьте код каждого из ваших модулей в соответствующие ячейки ниже.

**Строку, начинающуюся на %%writefile... стирать запрещено**

### **Содержимое модуля \_\_init__.py:**

In [1]:
!mkdir geo_transform

In [2]:
%%writefile geo_transform/__init__.py

"""
Публичный API пакета geo_transform.
Экспортируем классы и совместимые функции.
"""

from .transformations import CoordinateTransformer, cartesian_to_spherical, spherical_to_cartesian
from .utils import AngleUtils, deg_to_rad, rad_to_deg
from .file_operations import FileHandler, read_coordinates_from_file, write_results_to_file

__all__ = [
    "CoordinateTransformer",
    "cartesian_to_spherical",
    "spherical_to_cartesian",
    "AngleUtils",
    "deg_to_rad",
    "rad_to_deg",
    "FileHandler",
    "read_coordinates_from_file",
    "write_results_to_file",
]

Writing geo_transform/__init__.py


### **Содержимое модуля transformations.py:**

In [3]:
%%writefile geo_transform/transformations.py

# transformations.py
"""
Преобразования координат — теперь в объектно-ориентированном стиле.
Класс CoordinateTransformer инкапсулирует режим degrees (True/False).
Также предоставлены прежние функциональные обёртки для совместимости.
"""

from typing import Tuple
import math
from .utils import AngleUtils


class CoordinateTransformer:
    """
    Объект для преобразования координат между декартовой и сферической системами.

    Атрибуты:
        degrees (bool): если True — вход/выход углов в градусах, иначе в радианах.

    Методы:
        cartesian_to_spherical(x, y, z) -> (r, theta, phi)
        spherical_to_cartesian(r, theta, phi) -> (x, y, z)
    """

    def __init__(self, degrees: bool = False):
        self.degrees = bool(degrees)

    def cartesian_to_spherical(self, x: float, y: float, z: float) -> Tuple[float, float, float]:
        """
        Преобразование (x, y, z) -> (r, theta, phi).
        - r >= 0
        - theta: азимут в плоскости XY, отсчёт от +X, диапазон [0, 2*pi) или [0, 360) при degrees=True.
        - phi: угол от +Z (вниз), диапазон [0, pi] или [0, 180] при degrees=True.
        """
        r = math.sqrt(x * x + y * y + z * z)

        # Устойчивая формула для theta
        theta = 2 * math.atan2(y, x + math.sqrt(x * x + y * y))
        theta = theta % (2 * math.pi)

        # phi: угол от +Z
        rho = math.sqrt(x * x + y * y)
        phi = math.atan2(rho, z)  # возвращает в [0, pi]

        if self.degrees:
            theta = AngleUtils.rad_to_deg(theta)
            phi = AngleUtils.rad_to_deg(phi)

        return r, theta, phi

    def spherical_to_cartesian(self, r: float, theta: float, phi: float) -> Tuple[float, float, float]:
        """
        Преобразование (r, theta, phi) -> (x, y, z).
        Если self.degrees=True, theta и phi ожидаются в градусах.
        """
        if self.degrees:
            theta = AngleUtils.deg_to_rad(theta)
            phi = AngleUtils.deg_to_rad(phi)

        x = r * math.sin(phi) * math.cos(theta)
        y = r * math.sin(phi) * math.sin(theta)
        z = r * math.cos(phi)
        return x, y, z


# Функциональные обёртки для совместимости с прежним API:
_default_transformer = CoordinateTransformer(degrees=False)


def cartesian_to_spherical(x: float, y: float, z: float, degrees: bool = False) -> Tuple[float, float, float]:
    """
    Совместимая функция: если degrees=True — вернёт углы в градусах.
    """
    t = CoordinateTransformer(degrees=degrees)
    return t.cartesian_to_spherical(x, y, z)


def spherical_to_cartesian(r: float, theta: float, phi: float, degrees: bool = False) -> Tuple[float, float, float]:
    t = CoordinateTransformer(degrees=degrees)
    return t.spherical_to_cartesian(r, theta, phi)

Writing geo_transform/transformations.py


### **Содержимое модуля utils.py:**

In [4]:
%%writefile geo_transform/utils.py
# utils.py
"""
Утилиты для работы с углами.
Содержит класс AngleUtils и совместимые функции deg_to_rad / rad_to_deg.
"""

import math
from typing import Union

Number = Union[float, int]


class AngleUtils:
    """Статические методы для перевода градусов <-> радиан."""

    @staticmethod
    def deg_to_rad(deg: Number) -> float:
        """Преобразование градусов в радианы."""
        return float(deg) * math.pi / 180.0

    @staticmethod
    def rad_to_deg(rad: Number) -> float:
        """Преобразование радиан в градусы."""
        return float(rad) * 180.0 / math.pi


# Совместимость: доступны как функции на уровне модуля
def deg_to_rad(deg: Number) -> float:
    return AngleUtils.deg_to_rad(deg)


def rad_to_deg(rad: Number) -> float:
    return AngleUtils.rad_to_deg(rad)

Writing geo_transform/utils.py


### **Содержимое модуля file_operations.py:**

In [5]:
%%writefile geo_transform/file_operations.py
# file_operations.py
"""
Работа с файлами — класс FileHandler и совместимые функции.
FileHandler поддерживает чтение/запись списков троек (x,y,z) или (r,theta,phi).
"""

from typing import List, Tuple
import os

Coord = Tuple[float, float, float]


class FileHandler:
    """Класс для чтения и записи координат в текстовые файлы."""

    def __init__(self, encoding: str = "utf-8"):
        self.encoding = encoding

    def read_coordinates(self, path: str, system: str = "cartesian") -> List[Coord]:
        """
        Читает файл и возвращает список троек чисел.
        Формат: каждая непустая строка (не начинающаяся с '#') должна содержать
        как минимум три числа, разделённые пробелами/табами.
        system: семантический параметр ('cartesian' / 'spherical'), не влияет на парсинг.
        """
        coords: List[Coord] = []
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")

        with open(path, "r", encoding=self.encoding) as f:
            for raw in f:
                line = raw.strip()
                if not line or line.startswith("#"):
                    continue
                parts = line.split()
                if len(parts) < 3:
                    # пропускаем некорректные строки
                    continue
                try:
                    a, b, c = float(parts[0]), float(parts[1]), float(parts[2])
                    coords.append((a, b, c))
                except ValueError:
                    # пропускаем строки, где данные не числа
                    continue
        return coords

    def write_coordinates(self, path: str, coords: List[Coord], header: str = "") -> None:
        """
        Записывает список троек в текстовый файл (перезаписывает при наличии).
        """
        with open(path, "w", encoding=self.encoding) as f:
            if header:
                f.write(f"# {header}\n")
            for a, b, c in coords:
                f.write(f"{a} {b} {c}\n")


# Совместимость — старые функции с прежними именами
_default_handler = FileHandler()


def read_coordinates_from_file(path: str, system: str = "cartesian"):
    return _default_handler.read_coordinates(path, system=system)


def write_results_to_file(path: str, coords: List[Coord], header: str = ""):
    return _default_handler.write_coordinates(path, coords, header=header)

Writing geo_transform/file_operations.py


### **Содержимое модуля \_\_main__.py:**

In [6]:
%%writefile geo_transform/__main__.py

# __main__.py
"""
CLI-оболочка пакета (ООП-версия).
Примеры:
  python -m geo_transform cart2sph -i input.txt -o output.txt --deg
  python -m geo_transform sph2cart -i input_sph.txt -o output_cart.txt
"""

import argparse
import sys
from .transformations import CoordinateTransformer
from .file_operations import FileHandler


def main(argv=None):
    parser = argparse.ArgumentParser(description="geo_transform — преобразование координат (ООП)")
    parser.add_argument("mode", choices=["cart2sph", "sph2cart"],
                        help="cart2sph: декарт -> сферические; sph2cart: сферические -> декарт")
    parser.add_argument("-i", "--input", required=True, help="Входной файл с координатами")
    parser.add_argument("-o", "--output", required=True, help="Файл для записи результатов")
    parser.add_argument("--deg", action="store_true",
                        help="Интерпретировать/записывать углы в градусах (по умолчанию — радианы)")

    args = parser.parse_args(argv)

    handler = FileHandler()
    transformer = CoordinateTransformer(degrees=args.deg)

    system = "cartesian" if args.mode == "cart2sph" else "spherical"

    try:
        in_coords = handler.read_coordinates(args.input, system=system)
    except FileNotFoundError as e:
        print(f"Error: {e}", file=sys.stderr)
        sys.exit(2)

    results = []
    if args.mode == "cart2sph":
        for x, y, z in in_coords:
            r, theta, phi = transformer.cartesian_to_spherical(x, y, z)
            results.append((r, theta, phi))
        header = "r theta phi (theta,phi in degrees)" if args.deg else "r theta phi (radians)"
    else:
        for r, theta, phi in in_coords:
            x, y, z = transformer.spherical_to_cartesian(r, theta, phi)
            results.append((x, y, z))
        header = "x y z"

    handler.write_coordinates(args.output, results, header=header)
    print(f"Converted {len(results)} coordinate sets. Wrote to {args.output}")


if __name__ == "__main__":
    main()

Writing geo_transform/__main__.py


### **Содержимое модуля main.py (с импортом пакета и тестированием функций из него):**

In [9]:
# main.py
"""
Тестирование обновлённого ООП-пакета geo_transform.
Демонстрирует работу классов CoordinateTransformer, FileHandler и AngleUtils.
"""

from geo_transform import (
    CoordinateTransformer,
    FileHandler,
    AngleUtils,
    cartesian_to_spherical,      # функциональные обёртки (для совместимости)
    spherical_to_cartesian,
)


def main():
    print("Тест\n")

    # 1. Тест преобразования декартовых координат в сферические
    x, y, z = 1.0, 1.0, 1.0
    print(f"Декартовы координаты: x={x}, y={y}, z={z}")

    t_deg = CoordinateTransformer(degrees=True)
    r, theta, phi = t_deg.cartesian_to_spherical(x, y, z)
    print(f"→ Сферические координаты (через класс): r={r:.3f}, θ={theta:.3f}°, φ={phi:.3f}°\n")

    # 2. Тест обратного преобразования
    x2, y2, z2 = t_deg.spherical_to_cartesian(r, theta, phi)
    print(f"Обратное преобразование (через класс): x={x2:.3f}, y={y2:.3f}, z={z2:.3f}\n")

    # 3. Тест функциональных обёрток для совместимости
    print("Проверка функциональных обёрток:")
    r_f, th_f, ph_f = cartesian_to_spherical(x, y, z, degrees=True)
    print(f"cartesian_to_spherical(...): r={r_f:.3f}, th={th_f:.3f}, ph={ph_f:.3f}")

    x_f, y_f, z_f = spherical_to_cartesian(r_f, th_f, ph_f, degrees=True)
    print(f"spherical_to_cartesian(...): x={x_f:.3f}, y={y_f:.3f}, z={z_f:.3f}\n")

    # 4. Проверка перевода углов
    print("Проверка перевода углов:")
    deg = 180
    rad = AngleUtils.deg_to_rad(deg)
    print(f"{deg}° = {rad:.3f} рад, обратно = {AngleUtils.rad_to_deg(rad):.3f}°\n")

    # 5. Работа с файлами
    print("Тест работы с файлами:")
    handler = FileHandler()

    sample_coords = [(1.0, 0.0, 0.0), (0.0, 1.0, 1.0), (1.0, 1.0, 1.0)]
    handler.write_coordinates("input_cart.txt", sample_coords, header="x y z (декартовы координаты)")
    print("Файл input_cart.txt записан.")

    coords_from_file = handler.read_coordinates("input_cart.txt")
    print("Прочитанные координаты:")
    for c in coords_from_file:
        print(c)

    # 6. Преобразование координат из файла
    print("\nПроверка преобразований для координат из файла")
    sph_results = [t_deg.cartesian_to_spherical(x, y, z) for x, y, z in coords_from_file]

    handler.write_coordinates("output_sph.txt", sph_results, header="r theta phi (deg)")
    print("Файл output_sph.txt создан с результатами преобразований.")

    print("\nУспешно")


if __name__ == "__main__":
    main()

Тест

Декартовы координаты: x=1.0, y=1.0, z=1.0
→ Сферические координаты (через класс): r=1.732, θ=45.000°, φ=54.736°

Обратное преобразование (через класс): x=1.000, y=1.000, z=1.000

Проверка функциональных обёрток:
cartesian_to_spherical(...): r=1.732, th=45.000, ph=54.736
spherical_to_cartesian(...): x=1.000, y=1.000, z=1.000

Проверка перевода углов:
180° = 3.142 рад, обратно = 180.000°

Тест работы с файлами:
Файл input_cart.txt записан.
Прочитанные координаты:
(1.0, 0.0, 0.0)
(0.0, 1.0, 1.0)
(1.0, 1.0, 1.0)

Проверка преобразований для координат из файла
Файл output_sph.txt создан с результатами преобразований.

Успешно
